# install dependancies, takes around 45 seconds

Rendering Dependancies



In [ ]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
#!pip install gym pyvirtualdisplay 2>&1
#!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Pacman Dependancies

# Imports and Helper functions


In [23]:
import sys, os, copy
from pathlib import Path
from a2c.a2c import *
#from torch.distributions import Categorical
proj_folder = Path('.').absolute()
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Run Simulations!

In [24]:
import argparse, matplotlib.pyplot as plt, tqdm
def parse_a2c_arguments():
    # Command-line flags are defined here.
    parser = argparse.ArgumentParser()
    parser.add_argument('--env-name', dest='env_name', type=str,
                        default='CartPole-v0', help="Name of the environment to be run.")   # 'LunarLander-v2'
    parser.add_argument('--num-episodes', dest='num_episodes', type=int,
                        default=3500, help="Number of episodes to train on.")    # 3500
    parser.add_argument('--lr', dest='lr', type=float,
                        default=5e-4, help="The actor's learning rate.")
    parser.add_argument('--use_a2c', dest='use_a2c', type=bool,
                        default=True, help="Use A2C")
    parser.add_argument('--use_baseline', dest='use_baseline', type=bool,
                        default=True, help="Use baseline model")
    parser.add_argument('--baseline-lr', dest='baseline_lr', type=float,
                        default=5e-4, help="The actor's learning rate.")
    parser.add_argument('--critic-lr', dest='critic_lr', type=float,
                        default=1e-4, help="The critic's learning rate.")
    parser.add_argument('--n', dest='n', type=int,
                        default=100, help="The value of N in N-step A2C.")

    parser_group = parser.add_mutually_exclusive_group(required=False)
    parser_group.add_argument('--render', dest='render',
                              action='store_true',
                              help="Whether to render the environment.")
    parser_group.add_argument('--no-render', dest='render',
                              action='store_false',
                              help="Whether to render the environment.")
    parser.set_defaults(render=False)

    return parser.parse_known_args()[0]    #.parse_args()
args = parse_a2c_arguments()
DEBUG=False

In [25]:
history, res, A2C_net = main_a2c(args)

Configurations:Namespace(baseline_lr=0.0005, critic_lr=0.0001, env_name='CartPole-v0', lr=0.0005, n=100, num_episodes=3500, render=False, use_a2c=True, use_baseline=True)


  0%|                                                           | 0/5 [00:00<?, ?it/s]

Seed:0
Run type:A2C|Device:cuda
[Policy Evaluation]
The test reward for episode 0 is 25.1 with sd of 15.610573339887294.
[Policy Evaluation]
The test reward for episode 100 is 21.9 with sd of 11.099099062536562.
[Policy Evaluation]
The test reward for episode 200 is 24.55 with sd of 11.573569026017859.
[Policy Evaluation]
The test reward for episode 300 is 22.4 with sd of 13.093509842666327.
[Policy Evaluation]
The test reward for episode 400 is 28.0 with sd of 14.720733677368122.
[Policy Evaluation]
The test reward for episode 500 is 25.3 with sd of 13.255564869140809.
[Policy Evaluation]
The test reward for episode 600 is 23.45 with sd of 7.513155129504515.
[Policy Evaluation]
The test reward for episode 700 is 29.5 with sd of 19.971229306179428.
[Policy Evaluation]
The test reward for episode 800 is 42.8 with sd of 28.4035209085071.
[Policy Evaluation]
The test reward for episode 900 is 56.15 with sd of 33.835299614455906.
[Policy Evaluation]
The test reward for episode 1000 is 109.

  0%|                                                           | 0/5 [53:22<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 4.00 GiB total capacity; 3.51 GiB already allocated; 0 bytes free; 3.51 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
ks = np.arange(args.num_episodes/100)*100
avs = np.mean(res, axis=0)
maxs = np.max(res, axis=0)
mins = np.min(res, axis=0)

plt.fill_between(ks, mins, maxs, alpha=0.1)
plt.plot(ks, avs, '-o', markersize=1)

plt.xlabel('Episode', fontsize = 15)
plt.ylabel('Return', fontsize = 15)

if not os.path.exists('./plots'):
    os.mkdir('./plots')
print(A2C_net.type, args.n)
if A2C_net.type == 'A2C' or A2C_net.type == 2:
    plt.title("A2C Learning Curve for N = {}".format(args.n), fontsize = 24)
    plt.savefig("./plots/a2c_curve_N={}.png".format(args.n))
elif A2C_net.type == 'Baseline' or A2C_net.type == 1:
    plt.title("Baseline Reinforce Learning Curve".format(args.n), fontsize = 24)
    plt.savefig("./plots/Baseline_Reinforce_curve.png".format(args.n))
elif A2C_net.type == 'Reinforce' or A2C_net.type == 0: # Reinforce
    plt.title("Reinforce Learning Curve", fontsize = 24)
    plt.savefig("./plots/Reinforce_curve(2).png") 